In [ ]:
import ising_simulation_class as ising

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import FloatProgress ### Download ipywidgets in order to have progress bars, otherwise comment
from IPython.display import display  ### the lines marked with ipywidgets out
import time

from scipy.optimize import curve_fit


In [ ]:
MC_STEPS = 2000
DIM = 32
temp = 2.269 ### 2.269
J = 1
ITER = []
TIME = []
MAG = []

def tau(M):
#   unintegrated autocorrelation curve
    autocorr = auto_corr(M)    
#   integrated tau = 1 + 2*sum(G)
    return 1 + 2*np.sum(autocorr), autocorr

def auto_corr(M):
    kappa = 500
    auto_corr = np.zeros(kappa-1)
    mu = np.mean(M)
    for s in range(1,kappa-1):
        auto_corr[s] = np.mean( (M[:-s]-mu) * (M[s:]-mu) ) / np.var(M)
    return auto_corr


f = FloatProgress(min=0, max=MC_STEPS, description='Simulating...',) ### ipywidgtes
display(f) ### ipywidgtes
sim = ising.simulation(temperature=temp,c_strength=J,dimension=DIM)
start = time.time()
test = []
for i in range (MC_STEPS): ### DIM**2
    sim.metropolis()
    if i % 1 == 0: ### so after every MC STEP
        ITER.append(i)
        MAG.append(sim.get_magnetization())
        end = time.time()
        TIME.append(end-start)
    if i % 1 == 0: f.value+=1
f.close()

IAC, AUTO_CORR = tau(MAG)
print(IAC)

In [ ]:
def func(x,a,b):
    return a*np.exp(-x/b)

X = np.arange(len(AUTO_CORR))
x_fit = X[1:100]
y_fit = AUTO_CORR[1:100]

popt,pcov =curve_fit(func,x_fit,y_fit)
perr = np.sqrt(np.diag(pcov))

plt.figure(num=None, figsize=(4, 4), facecolor='w', edgecolor='k')
#plt.xlim(1,500)
#plt.ylim(-1,1)
plt.xlabel("MC Steps t")
plt.ylabel(r"$\Gamma_X(t)$")
plt.plot(X[1:],AUTO_CORR[1:],".",markersize=1,label="data")
fit1=r'$\tau_0=$'+str(round(popt[1],2))+r'$\pm$'+str(round(perr[1],2))
#fit1= r"fit function"
plt.plot(X, popt[0]*np.exp(-X/popt[1]),label=fit1, lw=1)
#plt.axhline(y=mag_avg, color='r', lw=1)
plt.legend(loc="best",prop={'size': 6.5})
plt.tight_layout()
plt.savefig('autocorrelation.png',dpi=300)
